<a href="https://colab.research.google.com/github/et-do/coffee-eda-spark/blob/main/cqi_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install pyspark

In [34]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import numpy as np

In [3]:
spark = SparkSession.builder\
  .master("local[*]")\
  .appName("learn-pyspark-1")\
  .getOrCreate()

spark

In [31]:
# read data, spark can't read driver-external files, so this pandas work around will do
from google.colab import drive
drive.mount('/content/gdrive')
csv = "/content/gdrive/MyDrive/cqi-eda/coffee_cleaned.csv"

df = spark.read.csv(csv,header=True,inferSchema=True, sep=',')

df.toPandas()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,_c0,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,...,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,0,Arabica,metad plc,Ethiopia,metad plc,None,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,0.0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,1,Arabica,metad plc,Ethiopia,metad plc,None,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,1.0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,2,Arabica,grounds for health admin,Guatemala,"""san marcos barrancas """"san cristobal cuch""",None,None,None,None,1600 - 1800 m,...,None,0.0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0
3,3,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,None,wolensu,None,yidnekachew debessa coffee plantation,1800-2200,...,Green,2.0,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0
4,4,Arabica,metad plc,Ethiopia,metad plc,None,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,2.0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,1334,Robusta,luis robles,Ecuador,robustasa,Lavado 1,our own lab,None,robustasa,None,...,Blue-Green,1.0,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN
1342,1335,Robusta,luis robles,Ecuador,robustasa,Lavado 3,own laboratory,None,robustasa,40,...,Blue-Green,0.0,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,40.0,40.0,40.0
1343,1336,Robusta,james moore,United States,fazenda cazengo,None,cafe cazengo,None,global opportunity fund,795 meters,...,None,6.0,"December 23rd, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,795.0,795.0,795.0
1344,1337,Robusta,cafe politico,India,None,None,None,14-1118-2014-0087,cafe politico,None,...,Green,1.0,"August 25th, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN


In [ ]:
# checkout schema
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Species: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Country.of.Origin: string (nullable = true)
 |-- Farm.Name: string (nullable = true)
 |-- Lot.Number: string (nullable = true)
 |-- Mill: string (nullable = true)
 |-- ICO.Number: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Altitude: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Producer: string (nullable = true)
 |-- Number.of.Bags: integer (nullable = true)
 |-- Bag.Weight: string (nullable = true)
 |-- In.Country.Partner: string (nullable = true)
 |-- Harvest.Year: string (nullable = true)
 |-- Grading.Date: string (nullable = true)
 |-- Owner.1: string (nullable = true)
 |-- Variety: string (nullable = true)
 |-- Processing.Method: string (nullable = true)
 |-- Aroma: double (nullable = true)
 |-- Flavor: double (nullable = true)
 |-- Aftertaste: double (nullable = true)
 |-- Acidity: double (nullable = true)
 |-- Bo

Looks like we've got a lot to work with here. First thing is cleaning up some of these headers by taking out the "."'s and removing any null heavy columns.

In [32]:
# Let's remove the "." from headers
# Anyone know how to loop through headers, this seems tedious
# presumeably something like:
# for col in df.columns:
#   df = df.withColumnRenamed(
#     col,
#     col without dot
#   )

df = df.withColumnRenamed("Country.of.Origin","origin_country") \
    .withColumnRenamed("Farm.Name","farm") \
    .withColumnRenamed("Lot.Number","lot") \
    .withColumnRenamed("ICO.Number","ico") \
    .withColumnRenamed("Number.of.Bags","bags") \
    .withColumnRenamed("Bag.Weight","bag_weight") \
    .withColumnRenamed("In.Country.Partner","country_partner") \
    .withColumnRenamed("Harvest.Year","harvest_year") \
    .withColumnRenamed("Grading.Date","grading_date") \
    .withColumnRenamed("Owner.1","owner_2") \
    .withColumnRenamed("Processing.Method","process_method")\
    .withColumnRenamed("Clean.Cup","clean_cup") \
    .withColumnRenamed("Cupper.Points","cupper_points") \
    .withColumnRenamed("Total.Cup.Points","total_cup_points") \
    .withColumnRenamed("Category.One.Defects","cat1_defect") \
    .withColumnRenamed("Category.Two.Defects","cat2_defect") \
    .withColumnRenamed("Certification.Body","cert_body") \
    .withColumnRenamed("Certification.Address","cert_address") \
    .withColumnRenamed("Certification.Contact","cert_contact")

df.printSchema()
print("that was horrid")

root
 |-- _c0: string (nullable = true)
 |-- Species: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- farm: string (nullable = true)
 |-- lot: string (nullable = true)
 |-- Mill: string (nullable = true)
 |-- ico: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Altitude: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Producer: string (nullable = true)
 |-- bags: integer (nullable = true)
 |-- bag_weight: string (nullable = true)
 |-- country_partner: string (nullable = true)
 |-- harvest_year: string (nullable = true)
 |-- grading_date: string (nullable = true)
 |-- owner_2: string (nullable = true)
 |-- Variety: string (nullable = true)
 |-- process_method: string (nullable = true)
 |-- Aroma: double (nullable = true)
 |-- Flavor: double (nullable = true)
 |-- Aftertaste: double (nullable = true)
 |-- Acidity: double (nullable = true)
 |-- Body: double (nullable = true)
 |-- Bala

Now to find Count of Null, None, NaN of All DataFrame Columns

In [35]:
# Can't have any "."'s in dataframe for this to work, hence the previous step
nulls = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])

nulls.show()

+---+-------+-----+--------------+----+----+----+---+-------+--------+------+--------+----+----------+---------------+------------+------------+-------+-------+--------------+-----+------+----------+-------+----+-------+----------+---------+---------+-------------+----------------+--------+-----------+-------+-----+-----------+----------+---------+------------+------------+-------------------+-------------------+--------------------+--------------------+
|_c0|Species|Owner|origin_country|farm| lot|Mill|ico|Company|Altitude|Region|Producer|bags|bag_weight|country_partner|harvest_year|grading_date|owner_2|Variety|process_method|Aroma|Flavor|Aftertaste|Acidity|Body|Balance|Uniformity|clean_cup|Sweetness|cupper_points|total_cup_points|Moisture|cat1_defect|Quakers|Color|cat2_defect|Expiration|cert_body|cert_address|cert_contact|unit_of_measurement|altitude_low_meters|altitude_high_meters|altitude_mean_meters|
+---+-------+-----+--------------+----+----+----+---+-------+--------+------+-----

Now going to remove any column with >20% missing values

In [36]:
# Setting threshold for nulls
total_rows = df.count()
null_threshold = .20
max_nulls = null_threshold*total_rows
max_nulls

# Defining our null finder/remover function
def drop_null_columns(df, null_threshold):
    """
    This function drops all columns which contain a threshold of null values.
    :param df: A PySpark DataFrame
    """
    total_rows = df.count()
    max_nulls = null_threshold*total_rows
    null_vals = nulls.collect()[0].asDict() # saves column names in list from null counter df in above cell
    to_drop = [k for k, v in null_vals.items() if v > max_nulls] # gets column names for all columns with null values above threshold
    df = df.drop(*to_drop) # drops all columns from to_drop list
    return df

# Call function on dataframe, passing through threshold parameter
df = drop_null_columns(df, .2)

# Validate new schema
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Species: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- ico: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Altitude: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Producer: string (nullable = true)
 |-- bags: integer (nullable = true)
 |-- bag_weight: string (nullable = true)
 |-- country_partner: string (nullable = true)
 |-- harvest_year: string (nullable = true)
 |-- grading_date: string (nullable = true)
 |-- owner_2: string (nullable = true)
 |-- Variety: string (nullable = true)
 |-- process_method: string (nullable = true)
 |-- Aroma: double (nullable = true)
 |-- Flavor: double (nullable = true)
 |-- Aftertaste: double (nullable = true)
 |-- Acidity: double (nullable = true)
 |-- Body: double (nullable = true)
 |-- Balance: double (nullable = true)
 |-- Uniformity: double (nullable = true)
 |-- clean_cup: double (nullable = 

Looks like our null heavy columns were dropped, now lets update our schema.
This will be a chance to re-type any columns as well as drop any remaining unnecessary columns from our analysis

In [ ]:
# update schema

data_schema = [
              StructField('_c0', IntegerType(), True),
              StructField('Lot.Number', IntegerType(), True),
              StructField('ICO.Number', IntegerType(), True),
              StructField('Altitude', FloatType(), True),
              StructField('Bag.Weight', FloatType(), True),
              StructField('Harvest.Year', DateType(), True),
              StructField('Grading.Date', DateType(), True),
              ]

final_struc = StructType(fields = data_schema)

df = spark.read.csv(
    csv_file, 
    header=True, 
    sep=',', 
    schema=final_struc)
